In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_2025_04_08_12_24.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.067606
1,star_graph_n7,0.0,0.039983
2,star_graph_n7,0.0,0.185797
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,0.0,0.027291


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.067606,0.067606,0.004571,1.0,0.067606
1,star_graph_n7,0.0,0.039983,0.039983,0.001599,1.0,0.039983
2,star_graph_n7,0.0,0.185797,0.185797,0.034520,1.0,0.185797
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,0.0,0.027291,0.027291,0.000745,1.0,0.027291
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
5276,graph_powerlaw_cluster_graph_n7,1.0,1.031464,0.031464,0.000990,0.0,0.031464
5277,graph_powerlaw_cluster_graph_n7,3.0,3.356488,0.356488,0.127084,0.0,0.118829
5278,graph_powerlaw_cluster_graph_n7,1.0,1.042522,0.042522,0.001808,0.0,0.042522


In [9]:
df['RE'].sum()/len(df)

0.07655307888153164

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1767,95.704980,0.054162,0.046698
1,2.0,1639,147.911891,0.090245,0.060929
2,1.0,870,107.122797,0.123130,0.027215
3,4.0,685,43.379335,0.063327,0.108895
4,0.0,253,2.383830,0.009422,0.000930
5,5.0,63,6.944688,0.110233,0.449959
6,6.0,3,0.752737,0.250912,2.437432


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,282.758431,0.072372,0.059280
1,graph_powerlaw_cluster_graph_n7,1350,119.261384,0.088342,0.059078
2,star_graph_n7,23,2.180441,0.094802,0.225121


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3781,282.102368,0.074611,0.061249
1,graph_powerlaw_cluster_graph_n7,1240,118.687731,0.095716,0.064306
2,star_graph_n7,6,1.026328,0.171055,0.830606


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,438,43.598876,0.099541,0.072548
1,graph_powerlaw_cluster_graph_n7,1.0,326,46.366049,0.142227,0.036229
2,graph_powerlaw_cluster_graph_n7,3.0,319,20.168427,0.063224,0.064021
3,graph_powerlaw_cluster_graph_n7,4.0,141,7.186305,0.050967,0.067478
4,graph_powerlaw_cluster_graph_n7,0.0,110,0.573653,0.005215,0.000149
5,graph_powerlaw_cluster_graph_n7,5.0,15,1.068417,0.071228,0.198858
6,graph_powerlaw_cluster_graph_n7,6.0,1,0.299657,0.299657,3.232596
7,graph_random_regular_graph_n7_d4,3.0,1445,75.337444,0.052137,0.042862
8,graph_random_regular_graph_n7_d4,2.0,1200,104.006221,0.086672,0.056424
9,graph_random_regular_graph_n7_d4,4.0,544,36.193030,0.066531,0.119630
